In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("sample").master("local[*]").getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [3]:
cnts_prod_mst_20210205 = spark.read.format("csv").option("header","true").load("C:/Users/b2en/Downloads/Spark 실습/homework_2/cnts_prod_mst_20210205.csv")
cnts_prod_mst_20210206 = spark.read.format("csv").option("header","true").load("C:/Users/b2en/Downloads/Spark 실습/homework_2/cnts_prod_mst_20210206.csv")
cnts_prod_mst_20210207 = spark.read.format("csv").option("header","true").load("C:/Users/b2en/Downloads/Spark 실습/homework_2/cnts_prod_mst_20210207.csv")
cnts_prod_mst_20210208 = spark.read.format("csv").option("header","true").load("C:/Users/b2en/Downloads/Spark 실습/homework_2/cnts_prod_mst_20210208.csv")
vod_wat_day_20210206 = spark.read.format("csv").option("header","true").load("C:/Users/b2en/Downloads/Spark 실습/homework_2/vod_wat_day_20210206.csv")
vod_wat_day_20210207 = spark.read.format("csv").option("header","true").load("C:/Users/b2en/Downloads/Spark 실습/homework_2/vod_wat_day_20210207.csv")

In [4]:
from pyspark.sql.functions import *
from pyspark.sql.window import *

In [28]:
cnts_prod_mst_20210205.withColumn("DATA_END_DT",coalesce(lead(col("STRD_DT")).over(Window.orderBy("STRD_DT")),lit('99991231')))\
                      .withColumn("DATA_STA_DT",col("STRD_DT"))\
                      .select("CNTS_PROD_ID","DATA_STA_DT","DATA_END_DT","CNTS_PROD_NM","CNTS_PROD_TYP_CD","PROD_PRC","SALE_PRC","USE_YN")

CNTS_PROD_ID,DATA_STA_DT,DATA_END_DT,CNTS_PROD_NM,CNTS_PROD_TYP_CD,PROD_PRC,SALE_PRC,USE_YN
PM50773595,20210205,99991231,런닝맨_21.01.17.일_SBS,10,1500,1500,Y


In [29]:
cnts_prod_mst_20210205.union(cnts_prod_mst_20210206)\
                      .withColumn("DATA_END_DT",coalesce(lead(col("STRD_DT")).over(Window.orderBy("STRD_DT")),lit('99991231')))\
                      .withColumn("DATA_STA_DT",col("STRD_DT"))\
                      .select("CNTS_PROD_ID","DATA_STA_DT","DATA_END_DT","CNTS_PROD_NM","CNTS_PROD_TYP_CD","PROD_PRC","SALE_PRC","USE_YN")

CNTS_PROD_ID,DATA_STA_DT,DATA_END_DT,CNTS_PROD_NM,CNTS_PROD_TYP_CD,PROD_PRC,SALE_PRC,USE_YN
PM50773595,20210205,20210206,런닝맨_21.01.17.일_SBS,10,1500,1500,Y
PM50773595,20210206,99991231,런닝맨_21.01.17.일_SBS,10,1500,1500,Y


In [30]:
cnts_prod_mst_20210205.union(cnts_prod_mst_20210206)\
                      .union(cnts_prod_mst_20210207)\
                      .withColumn("DATA_END_DT",coalesce(lead(col("STRD_DT")).over(Window.orderBy("STRD_DT")),lit('99991231')))\
                      .withColumn("DATA_STA_DT",col("STRD_DT"))\
                      .select("CNTS_PROD_ID","DATA_STA_DT","DATA_END_DT","CNTS_PROD_NM","CNTS_PROD_TYP_CD","PROD_PRC","SALE_PRC","USE_YN")

CNTS_PROD_ID,DATA_STA_DT,DATA_END_DT,CNTS_PROD_NM,CNTS_PROD_TYP_CD,PROD_PRC,SALE_PRC,USE_YN
PM50773595,20210205,20210206,런닝맨_21.01.17.일_SBS,10,1500,1500,Y
PM50773595,20210206,20210207,런닝맨_21.01.17.일_SBS,10,1500,1500,Y
PM50773595,20210207,99991231,런닝맨_21.01.17.일_SBS,10,1500,0,Y


In [36]:
vod_wat_day_20210206.join(cnts_prod_mst_20210206,vod_wat_day_20210206["CNTS_PROD_ID"] == cnts_prod_mst_20210206["CNTS_PROD_ID"],"inner")\
                    .withColumn("PYFR_YN",when(cnts_prod_mst_20210206["SALE_PRC"] >= 0,lit('Y')).otherwise(lit('N')))\
                    .select(vod_wat_day_20210206["STRD_DT"],vod_wat_day_20210206["SVC_MGMT_NUM"],vod_wat_day_20210206["CNTS_PROD_ID"],\
                            cnts_prod_mst_20210206["CNTS_PROD_NM"],vod_wat_day_20210206["WAT_CNT"],col("PYFR_YN"),cnts_prod_mst_20210206["SALE_PRC"])

STRD_DT,SVC_MGMT_NUM,CNTS_PROD_ID,CNTS_PROD_NM,WAT_CNT,PYFR_YN,SALE_PRC
20210206,7000000001,PM50773595,런닝맨_21.01.17.일_SBS,1,Y,1500
20210206,7000000002,PM50773595,런닝맨_21.01.17.일_SBS,1,Y,1500
20210206,7000000003,PM50773595,런닝맨_21.01.17.일_SBS,2,Y,1500
